In [1]:
import os
import podaac.subsetter.subset 
#import subset, transform_grouped_dataset
from podaac.subsetter.dimension_cleanup import remove_duplicate_dims
import shutil
import numpy as np
import xarray as xr
import netCDF4 as nc4
import random

In [6]:
# 'OMI-Aura_L2-OMSO2_2020m0116t1207-o82471_v003-2020m0223t142939.he5',
OMI_file_names = ['OMI-Aura_L2-OMBRO_2020m0116t1207-o82471_v003-2020m0116t182003.he5']
setrand = random.randint(1,1000)
OMI_copy_file = str(setrand)+'_OMI_copy_testing_1.he5'
OMI_copy_file_2 = str(setrand)+'_OMI_copy_testing_2.he5'
OMI_output_file = str(setrand)+'_OMI_output.he5'
for i in OMI_file_names:
    shutil.copyfile(os.path.join('tests/data','OMI', i),
                    os.path.join('mytest', OMI_copy_file))
    shutil.copyfile(os.path.join('tests/data','OMI', i),
                    os.path.join('mytest', OMI_copy_file_2))
    min_time='2020-01-16T12:30:00Z'
    max_time='2020-01-16T12:40:00Z'
    bbox = np.array(((-180, 180), (-90, 90)))
    nc_dataset, has_groups = podaac.subsetter.subset.h5file_transform(os.path.join('mytest', OMI_copy_file))
    args = {
        'decode_coords': False,
        'mask_and_scale': False,
        'decode_times': False
    }

    with xr.open_dataset(
            xr.backends.NetCDF4DataStore(nc_dataset),
            **args
    ) as input_dataset:
        #print (dataset)
        print (input_dataset['__HDFEOS__SWATHS__OMI Total Column Amount BrO__Geolocation Fields__Time'].attrs)
    podaac.subsetter.subset.subset(
        file_to_subset=os.path.join('mytest', OMI_copy_file_2),
        bbox=bbox,
        output_file=os.path.join('mytest', OMI_output_file),
        min_time=min_time,
        max_time=max_time
    )
    
    output_ncdataset = nc4.Dataset(os.path.join('mytest', OMI_output_file))
    cut_set = output_ncdataset.groups['HDFEOS'].groups['SWATHS'].groups['OMI Total Column Amount BrO'].groups['Geolocation Fields']
    xrout_dataset = xr.open_dataset(xr.backends.NetCDF4DataStore(cut_set))
    outattrs = xrout_dataset['Time'].attrs
    

{'_FillValue': -1e+30, 'Title': 'Time in TAI units', 'Units': 's', 'UniqueFieldDefinition': 'Aura-Shared', 'Offset': 0.0, 'ScaleFactor': 1.0, 'MissingValue': -1e+30, 'ValidRange': array([0.e+00, 1.e+10])}
{'Title': 'Time in TAI units', 'Units': 's', 'UniqueFieldDefinition': 'Aura-Shared', 'Offset': 0.0, 'ScaleFactor': 1.0, 'MissingValue': -1e+30, 'ValidRange': array([0.e+00, 1.e+10])}
